In [1]:
%load_ext autoreload
%autoreload 1

%aimport f
%aimport t

framework loaded, version: 0.2


In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
from IPython.display import clear_output

import time

import numpy as np
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_colwidth = 999
pd.options.display.max_rows = 113

import matplotlib.pyplot as plt

import seaborn as sns
from sklearn.preprocessing import StandardScaler, PowerTransformer

from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture, BayesianGaussianMixture

RS = 35577 # global random state seed
data_path = './data-raw/'

In [3]:
df_data = f.from_pkl(f'data-processed/df_clean.pkl')

df_data.shape
df_data.head(10)


(97452, 9)

,make,model,year,price,transmission,mileage,fuelType,tax,engineSize
0,Audi,A1,2017,12500,Manual,15735,Petrol,150,1.4
1,Audi,A6,2016,16500,Automatic,36203,Diesel,20,2.0
2,Audi,A1,2016,11000,Manual,29946,Petrol,30,1.4
3,Audi,A4,2017,16800,Automatic,25952,Diesel,145,2.0
4,Audi,A3,2019,17300,Manual,1998,Petrol,145,1.0
5,Audi,A1,2016,13900,Automatic,32260,Petrol,30,1.4
6,Audi,A6,2016,13250,Automatic,76788,Diesel,30,2.0
7,Audi,A4,2016,11750,Manual,75185,Diesel,20,2.0
8,Audi,A3,2015,10200,Manual,46112,Petrol,20,1.4
9,Audi,A1,2016,12000,Manual,22451,Petrol,30,1.4


# Encode

In [135]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

## transmission & fuelType

In [136]:
df_enc1 = df_data.copy()

In [137]:
df_enc1['transmission'].value_counts()
df_enc1['fuelType'].value_counts()

t.ordinalEncode(df_enc1, ['transmission', 'fuelType'])

df_enc1['transmission'].value_counts()
df_enc1['fuelType'].value_counts()

Manual       55470
Semi-Auto    22295
Automatic    19687
Name: transmission, dtype: int64

Petrol    53978
Diesel    40417
Hybrid     3057
Name: fuelType, dtype: int64

1    55470
2    22295
0    19687
Name: transmission, dtype: int64

2    53978
0    40417
1     3057
Name: fuelType, dtype: int64

## make & model

In [138]:
df_audi = df_enc1.loc[df_enc1['make'] == 'Audi'].drop(columns='make').copy()

model_vc = df_audi['model'].value_counts()
model_vc

models_to_keep = model_vc[model_vc>10].index
df_audi = df_audi.loc[ df_audi['model'].isin(models_to_keep) ]
df_audi['model'].value_counts()

 A3     1917
 Q3     1379
 A4     1375
 A1     1337
 A5      874
 Q5      864
 Q2      810
 A6      748
 Q7      394
 TT      336
 A7      122
 A8      118
 Q8       69
 RS6      39
 RS3      33
 RS4      31
 RS5      28
 R8       28
 S3       18
 SQ5      16
 S4       12
 SQ7       8
 S8        4
 S5        3
 A2        1
 RS7       1
Name: model, dtype: int64

 A3     1917
 Q3     1379
 A4     1375
 A1     1337
 A5      874
 Q5      864
 Q2      810
 A6      748
 Q7      394
 TT      336
 A7      122
 A8      118
 Q8       69
 RS6      39
 RS3      33
 RS4      31
 RS5      28
 R8       28
 S3       18
 SQ5      16
 S4       12
Name: model, dtype: int64

In [139]:
t.oh_encode(df_audi, ['model'])

In [140]:
df_train = df_audi

In [144]:
df_train.head(3)
df_train.dtypes

,year,price,transmission,mileage,fuelType,tax,engineSize,model__ A1,model__ A6,model__ A4,model__ A3,model__ Q3,model__ Q5,model__ A5,model__ S4,model__ Q2,model__ A7,model__ TT,model__ Q7,model__ RS6,model__ RS3,model__ A8,model__ Q8,model__ RS4,model__ RS5,model__ R8,model__ SQ5,model__ S3
0,2017,12500,1,15735,2,150,1.4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2016,16500,0,36203,0,20,2.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2016,11000,1,29946,2,30,1.4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


year              int64
price             int64
transmission      int64
mileage           int64
fuelType          int64
tax               int64
engineSize      float64
model__ A1        int64
model__ A6        int64
model__ A4        int64
model__ A3        int64
model__ Q3        int64
model__ Q5        int64
model__ A5        int64
model__ S4        int64
model__ Q2        int64
model__ A7        int64
model__ TT        int64
model__ Q7        int64
model__ RS6       int64
model__ RS3       int64
model__ A8        int64
model__ Q8        int64
model__ RS4       int64
model__ RS5       int64
model__ R8        int64
model__ SQ5       int64
model__ S3        int64
dtype: object

In [145]:
X = df_train.drop('price', axis=1)
y = df_train['price']

In [146]:
X.shape
y.shape

(10548, 27)

(10548,)

# Train model

In [147]:
from sklearn.model_selection import KFold, StratifiedKFold, RepeatedStratifiedKFold
from lightgbm import LGBMRegressor

In [148]:
import itertools

def grid_exec(func, *args):
    return [func(*element) for element in itertools.product(*args)]

def grid_exec_callback(func, callback, *args):
    return [ callback(func(*element)) for element in itertools.product(*args)]

In [149]:
def get_stats_df(cv_results):
    ret_list = []
    
    for result in cv_results:
        model_params = result[1]
        model_params.pop('random_state', None)
        model_params.pop('silent', None)
        
        stats_ = np.array(result[4])
        
        ret_list.append(
            {
                'model' : result[0],
                # '#' : None,
                # '#1' : None,
                # '#2' : None,
                # '#3' : None, 
                'params' : str(model_params).strip('{').strip('}'),
                'n_folds' : result[2],    

#                 'min': stats_.min(),
#                 'mean': stats_.mean(),
#                 'max': stats_.max(),
#                 'std': stats_.std(),

#                 'folds_min': stats_.mean(axis=1).min(),
#                 'folds_mean': stats_.mean(axis=1).mean(),
#                 'folds_max': stats_.mean(axis=1).max(),
#                 'folds_std': stats_.mean(axis=1).std(),
                'M1': stats_.mean(axis=0)[0],
                'M1_std': stats_.std(axis=0)[0],
                'M2': stats_.mean(axis=0)[1],
                'M2_std': stats_.std(axis=0)[1],
                'M3': stats_.mean(axis=0)[2],
                'M3_std': stats_.std(axis=0)[2],
                
                'time': round(result[3], 1), 
            })
    df = pd.DataFrame(ret_list)
    return df
    # return df.sort_values(by=['mean', 'std'], ascending=[False, True]).reset_index(drop=True)

def display_stats(df_stats):
    styler = df_stats.style
    
    styler.format('{:,.1f}', 'time')
    styler.bar(subset='time')
    
    styler.background_gradient(cmap='Oranges', subset=[c for c in df_stats.columns if c[0] == '#'])
    
    # styler.format('{:,.0f}', ['min', 'mean', 'max', 'std', 'folds_min', 'folds_mean', 'folds_max', 'folds_std'])
    for c in ['M1', 'M2', 'M3']:
        styler.background_gradient(cmap='Blues', subset=c, gmap=df_stats[c].rank())

#     for c in ['min', 'mean', 'max', 'std']:
#         styler.background_gradient(cmap='Blues', subset=c, gmap=df_stats[c].rank())
        
#     for c in ['folds_min', 'folds_mean', 'folds_max', 'folds_std']:
#         styler.background_gradient(cmap='Greens', subset=c, gmap=df_stats[c].rank())
    
    display(styler)
    # return styler
    
# def display_stats_v1(df_stats):
#     display(df_stats.style\
#             .format('{:,.3f}', df_stats.columns[5:])\
#             .format('{:,.1f}', df_stats.columns[-1])\
#             .background_gradient(cmap='Reds', subset=['#1', '#2'], axis=0)\
#             .background_gradient(cmap='Blues', subset=df_stats.columns[6:10], axis=0)\
#             .background_gradient(cmap='Greens', subset=df_stats.columns[10:-1], axis=0)\
#            )

In [ ]:
mean_squared_error(

In [156]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def root_mean_squared_error(y_true, y_pred):
    return mean_squared_error(y_true, y_pred, squared=False)
    

In [151]:
all_trained_models = []

def cross_val(model_and_params, X, y, eval_metric, n_folds = 5, n_fold_repeats = 1):
    np.random.seed(RS)
    get_random = lambda  : np.random.randint(1, 2**16)
    cv_scores = []
    n_folds_completed = 0
    
    start_time = time.perf_counter()
    
    # cv = RepeatedStratifiedKFold(n_splits=n_folds, n_repeats=n_fold_repeats, random_state=get_random())
    cv = KFold(n_splits=n_folds)
    for i_fold, (idx_train, idx_test) in enumerate(cv.split(X)):
        X_train, y_train = X.iloc[idx_train], y.iloc[idx_train].values
        X_test, y_test = X.iloc[idx_test], y.iloc[idx_test].values

        

        constructor, params_dic = model_and_params

        if 'random_state' in params_dic.keys():
            params_dic['random_state'] = get_random()

        model = constructor(**params_dic)
        all_trained_models.append(model)

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        scores = [ metric(y_test, y_pred) for metric in eval_metric ]

        # score = mean_squared_error(y_pred, y_test, squared=False)
        # score = mean_absolute_error(y_pred, y_test)
        # score = model.score(X_test, y_test, )

        cv_scores.append(scores)
        n_folds_completed += 1

    model_name = model.__class__.__name__
    model_params = params_dic.copy()
        
    total_elapsed_time = time.perf_counter() - start_time
    return model_name, model_params, n_folds_completed, total_elapsed_time, cv_scores

In [157]:
all_results = []
def on_1_completed(results):
    all_results.append(results)
    clear_output(wait=True)
    get_stats_df(all_results)
    display_stats(get_stats_df(all_results))

grid_func = lambda factory, n_folds, n_fold_repeats: cross_val(factory, X, y, metric_list, n_folds, n_fold_repeats)

n_est_list = [1000]

all_models = [
    *[ (LGBMRegressor,         {'random_state': RS, 'n_estimators': n_e}) for n_e in n_est_list ],
]

metric_list = [ mean_absolute_error, root_mean_squared_error, r2_score]

n_folds_list = [8]
# n_fold_repeats_list = [1]

_ = grid_exec_callback(grid_func, on_1_completed, 
                       all_models,
                       n_folds_list, 
                       n_fold_repeats_list)

,model,params,n_folds,M1,M1_std,M2,M2_std,M3,M3_std,time
0,LGBMRegressor,'n_estimators': 1000,8,1851.579379,278.309158,2798.710982,508.410322,0.936128,0.016793,6.7
